## необходимые функции и пояснения

By Поляков Борис Георгиевич КМБО-03-23 thewaterbox@yandex.ru

In [ ]:
include("../KingBot.jl")

Импортированные объекты:

1. Структура-обертка, фигура "Король" на шахматном поле
    KingBot содержит:
    * robot::HorizonSideRobots.Robot - Инстанция робота пакета HorizonSideRobots  
    * rx::Integer - изменение по оси x относительно старта или очистки буфера  
    * ry::Integer - изменение по оси y относительно старта или очистки буфера  
    * movesBuffer::Array{MoveLog} - буфер, содержащий историю движений робота  
    * traceMove::Bool - Это логическое поле, указывающее, следует ли отслеживать перемещения робота.  
    * moveFunction::Function - определяющая движения функция для совместимости с другими пакетами или особыми типами движения

    <br>
2. Буфер, состоит из структур MoveLog
    * direction::Union{Nothing, HorizonSideRobots.HorizonSide, Diagonal} - направление
    * steps::Union{Nothing, Integer} - количество шагов

    <br>

3. Аналоги (обертки) функций робота из пакета HorizonSideRobots для фигуры KingBot, (move!, isborder, putmarker! и др.)

4. Функции поворота
    * inverse(...) - разворачивает направление заданного объекта на 180 градусов
    * turnCounterClockwise(side::HorizonSide) - разворачивает переданное направление один раз против часовой стрелки
    * turnClockwise(side::HorizonSide) = HorizonSide(mod(Int(side)-1,4)) - разворачивает переданное направление один раз по часовой стрелке

    <br>
5. Служебные функции
    * clearBuffer! - очищает буфер движений в инстанции KingBot
    * clearLogging! - очищает помимо буфера релятивные координаты
    * moveTill! - двигает робота в заданном направлении до наступления условия
    * checkAllBorders - возвращает вектор из границ вокруг робота (без диагоналей)
    * checkAllUnblockedPaths - возвращает вектор из свободных направлений вокруг робота (без диагоналей)

    <br>
6. Функция обхода
getNextPerimeterSide - возвращает следующую сторону для обхода фигуры по часовой/против часовой стрелки

    Аргументы:
    * bot::KingBot - инстанция фигуры KingBot
    * dominatingSide - сторона, указывающая на расположение заданной фигуры относительно робота в начале
    * rotatingFunction::Function=turnClockwise - функция вращения, по умолчанию по часовой

Идея логики выбора стороны:

Однозначно определено направление движения по функции вращения если робот стоит у 3 или 2 стен  
Если робот не сдвигался вокруг фигуры - робот ориентируется на переданную сторону и делает выбор по ней (например если две фигуры стоят вплотную)  
В остальных случаях рассматривается последнее движение робота и его поворот, если нельзя провернуть - робот продолжает движение по последнему ходу

### Задание 1. Определить внутри или снаружи

In [ ]:
function isInside!(bot::KingBot, side::HorizonSide, rotatingFunction::Function=turnClockwise)
    moveTill!(bot, side, isborder, [bot, side])
    clearLogging!(bot)
    
    toCheck = checkAllBorders(bot)
    up, left, down, right = false, false, false, false
    maxY, maxX, minY, minX = 0, 0, 0, 0

    # these inner functions arent supposed to be used anywhere else, they are sort of Syntactic sugar
    # updates toCheck based on the bot's last moves and the type of rotatingFunction
    function excludeSides!()
        if isempty(bot.movesBuffer)
            side = rotatedNextMove()
            setdiff!(toCheck, [side])
        else
            side = rotatingFunction(last(bot.movesBuffer).direction)
            (bot.rx == 0 && bot.ry == 0) && setdiff!(toCheck, [side])
        end

        if !isempty(bot.movesBuffer) && (bot.rx == 0 && bot.ry == 0) && length(checkAllBorders(bot)) == 3
            empty!(toCheck)
        end
    end

    # the function returns next side to move around object after rotating it 
    function rotatedNextMove()
        return rotatingFunction(getNextPerimeterSide(bot, side, rotatingFunction))
    end

    # the function returns next side to move around object without rotating it 
    function nextMove()
        return getNextPerimeterSide(bot, side,rotatingFunction)
    end

    while true
        # update toCheck
        excludeSides!()

        # update flags to determine bot's position
        if !isempty(checkAllBorders(bot)) && isborder(bot, rotatedNextMove())
            maxY = max(maxY, bot.ry)
            minY = min(minY, bot.ry)
            maxX = max(maxX, bot.rx)
            minX = min(minX, bot.rx)
            if bot.ry >= maxY up = !isborder(bot, inverse(North)) end
            if bot.ry <= minY down = !isborder(bot, inverse(South)) end
            if bot.rx >= maxX right = !isborder(bot, inverse(East)) end
            if bot.rx <= minX left = !isborder(bot, inverse(West)) end
        end

        # if the toCheck is empty, the movement around structure is completed
        if isempty(toCheck)
            return any([up, left, down, right])
        end
        
        move!(bot, nextMove())

    end
end

Ключевые моменты функции:

1. Вектор toCheck нужен чтобы отслеживать остановку обхода не только по координатам (то, что робот попал в (0,0) второй раз не означает, что обход закончен, поэтому нужна дополнительная информация). Инициализируется перегородками вокруг робота в стартовой позиции

2. up, left, down, right, maxY, maxX, minY, minX отслеживают координаты и состояние робота в крайних точках лабиринта.
Если к границе можно прийти изнутри (из противоположного направления) - значит робот находится внутри. Очевидно, что если робот снаружи - таких точек не найдется. Углы игнорируются.

Перед рабочим циклом: Дойти до заданной формации и обнулить собранную информацию (если робот начинает не у стенки формации)
 
3. Общий вид рабочего цикла:
- обновить toCheck
- обновить переменные up, left, down, right и учет краев
- Проверить условия прерывания цикла
- Сделать следующее движение по периметру

2. Определить площадь фигуры

In [ ]:
function calculateArea!(bot::KingBot, side::HorizonSide, rotatingFunction::Function=turnClockwise)
    moveTill!(bot, side, isborder, [bot, side])
    clearLogging!(bot)

    toCheck = checkAllBorders(bot)
    cnt = 0
    # these inner functions arent supposed to be used anywhere else, they are sort of Syntactic sugar
    # check the left side and update counter
    function checkLeft()
        if isborder(bot, West)
            # putmarker!(bot)
            # println("+ $(bot.rx) ($(bot.rx), $(bot.ry)) ")
            cnt += bot.rx
        end
    end
    # check the right side and update counter
    function checkRight()
        if isborder(bot, East)
            # println("- $(bot.rx + 1) ($(bot.rx), $(bot.ry)) ")
            # putmarker!(bot)
            # add correction
            cnt -= bot.rx + 1
        end
    end

    # handle which side to update depending on the rotatingFunction
    function checkBoth!(side::HorizonSide)
        if side in (rotatingFunction(West), East) checkRight()
        else checkLeft()
        end
    end
    
    # the function returns next side to move around object after rotating it 
    function rotatedNextMove()
        return rotatingFunction(getNextPerimeterSide(bot, side, rotatingFunction))
    end
    
    # the function returns next side to move around object without rotating it 
    function nextMove()
        return getNextPerimeterSide(bot, side,rotatingFunction)
    end

    function lastMove()
        if isempty(bot.movesBuffer)
            return nextMove()
        end
        return last(bot.movesBuffer).direction
    end

    # updates toCheck based on the bot's last moves and the type of rotatingFunction
    function excludeSides!()
        # println(toCheck)
        if isempty(bot.movesBuffer)
            side = rotatedNextMove()
            if length(toCheck) == 3
                checkLeft()
                checkRight()
            end
            setdiff!(toCheck, [side])
        else
            side = rotatingFunction(last(bot.movesBuffer).direction)
            if (bot.rx == 0 && bot.ry == 0)
                if (length(checkAllBorders(bot)) != 3)
                    checkBoth!(lastMove())
                end
                setdiff!(toCheck, [side])
            end
        end

        if !isempty(bot.movesBuffer) && (bot.rx == 0 && bot.ry == 0) && length(checkAllBorders(bot)) == 3
            empty!(toCheck)
        end
    end

    while true
        # update toCheck
        excludeSides!()
        
        # if the toCheck is empty, the movement around structure is completed
        if isempty(toCheck)
            return cnt
        end
        
        # update counter
        if isborder(bot, rotatedNextMove()) && !isempty(bot.movesBuffer) && (bot.rx != 0 || bot.ry != 0)
            checkBoth!(lastMove())
            if length(checkAllBorders(bot)) == 3
                checkBoth!(inverse(lastMove()))
            end
        end
        
        move!(bot, nextMove())
    end
end

Ключевые моменты функции:

1. Вектор toCheck нужен чтобы отслеживать остановку обхода не только по координатам (то, что робот попал в (0,0) второй раз не означает, что обход закончен, поэтому нужна дополнительная информация). Инициализируется перегородками вокруг робота в стартовой позиции 

2. Робот смотрит на некоторую сторону (в зависимости от вращающей функции) и обновляет по ней счетчик. Обновление происходит в виде обработки разности координат X границы и начального положения с корректировкой. Операции делегирует вызов функции checkBoth!

Перед рабочим циклом: Дойти до заданной формации и обнулить собранную информацию (если робот начинает не у стенки формации)

3. Общий вид рабочего цикла:
- Обновить toCheck
- Проверить условия прерывания цикла
- Осмотреть границы (в зависимости от вращающей функции) и обновить счетчик
- Сделать следующее движение по периметру

## Тесты 1 функции:

In [ ]:
# Для показа работы функции в данном блоке
function testInside!(situationFile, side, rotatingFunction=turnClockwise)
    bot = KingBot(situationFile)
    sleep(0.5) # чтобы можно было увидеть откуда робот начинает
    if isInside!(bot, side, rotatingFunction)
        println("Внутри")
    else 
        println("Снаружи")
    end
end

In [ ]:
testInside!("Situations/1.sit", North)

In [ ]:
testInside!("Situations/1.1.sit", North, turnCounterClockwise)

In [ ]:
testInside!("Situations/1.2.sit", South)

In [ ]:
testInside!("Situations/2.sit", South, turnCounterClockwise)

In [ ]:
testInside!("Situations/3.sit", North)

In [ ]:
testInside!("Situations/3.1.sit", South)

## Тесты 2 функции

Note: Эта же функция дает информацию, находится ли робот внутри или снаружи.  
Площадь равна полученному значению по модулю. Если функция выдает отрицательное значение - робот внутри формации, если положительное - снаружи

In [ ]:
# Для показа работы функции в данном блоке
function testArea!(situationFile, side, rotatingFunction=turnClockwise)
    bot = KingBot(situationFile)
    sleep(0.5) # чтобы можно было увидеть откуда робот начинает
    println(calculateArea!(bot, side, rotatingFunction))
end

In [ ]:
testArea!("Situations/1.sit", North)
#Answer = 18

In [ ]:
testArea!("Situations/1.1.sit", North, turnCounterClockwise)
#Answer = 18

In [ ]:
testArea!("Situations/1.2.sit", South)
#Answer = -18 (Внутри)

In [ ]:
testArea!("Situations/2.sit", South)
#Answer = 5

In [ ]:
testArea!("Situations/3.sit", South)
#Answer = -17 (Внутри)

In [ ]:
testArea!("Situations/3.1.sit", South)
#Answer = 17